In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import dask.dataframe as ddf
from pandas import Series, DataFrame
%matplotlib inline

In [2]:
import seaborn as sn
import plotly.express as px

### Request for Census Tract Level Population data from Census API

In [3]:
stateCode = ['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13', '15', '16',
             '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29',
            '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', 
             '44', '45', '46', '47', '48', '49', '50', '51', '53', '54', '55', '56']

In [4]:
len(stateCode)

51

In [5]:
import requests
import pandas as pd
from datetime import datetime
import time
import json
from io import StringIO

In [6]:
def collectAll_county(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs5?get=NAME,'+ variables + '&for=county:*&in=state:*&key=250f58e35283d400e4185b197e3abe9021079afe'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

#### Total population: B01003_001E

In [7]:
year = '2015'
variables = 'B01003_001E'

In [8]:
pop_17 = collectAll_county(year,variables)
pop_17.head()

['NAME', 'B01003_001E', 'state', 'county']


,county_name,B01003_001E,state code,county code
0,"Childress County, Texas",7059,48,075
1,"Comal County, Texas",119632,48,091
2,"Houston County, Texas",22949,48,225
3,"Navarro County, Texas",48118,48,349
4,"Scurry County, Texas",17238,48,415


In [9]:
pop_17.shape

(3220, 4)

#### 1.per capita income: B19301_001E

In [10]:
variables = 'B19301_001E'
df1 = collectAll_county(year,variables)

['NAME', 'B19301_001E', 'state', 'county']


In [11]:
df1.shape

(3220, 4)

In [12]:
df1 = df1.rename(columns = {'B19301_001E': 'per_capita_income'})
df1['fips'] = df1['state code'] + df1['county code']
df1.head()

,county_name,per_capita_income,state code,county code,fips
0,"Childress County, Texas",19861,48,075,48075
1,"Comal County, Texas",32838,48,091,48091
2,"Houston County, Texas",17570,48,225,48225
3,"Navarro County, Texas",20697,48,349,48349
4,"Scurry County, Texas",24584,48,415,48415


#### 2.Percent of Population Below Poverty Level: (C17002_002E+C17002_003E)/C17002_001E

In [13]:
variables = 'C17002_002E,C17002_003E,C17002_001E'
df2 = collectAll_county(year,variables)

['NAME', 'C17002_002E', 'C17002_003E', 'C17002_001E', 'state', 'county']


In [14]:
df2['C17002_002E'] = df2['C17002_002E'].astype('int')
df2['C17002_003E'] = df2['C17002_003E'].astype('int')
df2['C17002_001E'] = df2['C17002_001E'].astype('int')

In [15]:
df2['percent_below_poverty'] = ((df2['C17002_002E'] + df2['C17002_003E'])/df2['C17002_001E'])*100

In [16]:
df2['fips'] = df2['state code'] + df2['county code']
df2.head()

,county_name,C17002_002E,C17002_003E,C17002_001E,state code,county code,percent_below_poverty,fips
0,"Childress County, Texas",339,838,5823,48,075,20.212949,48075
1,"Comal County, Texas",4149,6274,118403,48,091,8.802986,48091
2,"Houston County, Texas",1768,3150,19663,48,225,25.011443,48225
3,"Navarro County, Texas",3749,6045,47225,48,349,20.739015,48349
4,"Scurry County, Texas",420,1107,15490,48,415,9.857973,48415


#### 3.Percent of Population 25+ with Less than a 12th Grade Education: (B15002_003E,B15002_004E,B15002_005E,B15002_003E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E)/B15003_001E

In [17]:
def collectAll_county2(year, variables):
    basic_url = 'https://api.census.gov/data/' + year + '/acs/acs1?get=NAME,'+ variables + '&for=county:*&in=state:*'
    r = requests.get(basic_url)
    x = r.json()
    print(x[0])
    variable_list = variables.split(',')
    column_list = ['county_name']
    for item in variable_list:
        column_list.append(item)
    column_list.append('state code')
    column_list.append('county code')
    df = pd.DataFrame(x[1:])
    df.columns = column_list
    return df

In [18]:
variables = 'B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,B15002_021E,B15002_022E,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E'
df3 = collectAll_county(year,variables)

['NAME', 'B15002_003E', 'B15002_004E', 'B15002_005E', 'B15002_006E', 'B15002_007E', 'B15002_008E', 'B15002_009E', 'B15002_010E', 'B15002_020E', 'B15002_021E', 'B15002_022E', 'B15002_023E', 'B15002_024E', 'B15002_025E', 'B15002_026E', 'B15002_027E', 'B15002_001E', 'state', 'county']


In [19]:
df3.shape

(3220, 20)

In [20]:
def changeInteger(value):
    if value != None:
        return int(value)
    else:
        return np.nan

In [21]:
df3['B15002_003E'] = df3['B15002_003E'].apply(changeInteger)
df3['B15002_004E'] = df3['B15002_004E'].apply(changeInteger)
df3['B15002_005E'] = df3['B15002_005E'].apply(changeInteger)
df3['B15002_006E'] = df3['B15002_006E'].apply(changeInteger)
df3['B15002_007E'] = df3['B15002_007E'].apply(changeInteger)
df3['B15002_008E'] = df3['B15002_008E'].apply(changeInteger)
df3['B15002_009E'] = df3['B15002_009E'].apply(changeInteger)
df3['B15002_010E'] = df3['B15002_010E'].apply(changeInteger)
df3['B15002_020E'] = df3['B15002_020E'].apply(changeInteger)
df3['B15002_021E'] = df3['B15002_021E'].apply(changeInteger)
df3['B15002_022E'] = df3['B15002_022E'].apply(changeInteger)
df3['B15002_023E'] = df3['B15002_023E'].apply(changeInteger)
df3['B15002_024E'] = df3['B15002_024E'].apply(changeInteger)
df3['B15002_025E'] = df3['B15002_025E'].apply(changeInteger)
df3['B15002_026E'] = df3['B15002_026E'].apply(changeInteger)
df3['B15002_027E'] = df3['B15002_027E'].apply(changeInteger)
df3['B15002_001E'] = df3['B15002_001E'].apply(changeInteger)

In [22]:
df3['percent_no_highschool_diploma'] = ((df3['B15002_003E']+df3['B15002_004E']+df3['B15002_005E']+df3['B15002_006E']+df3['B15002_007E']+df3['B15002_008E']+df3['B15002_009E']
                                       +df3['B15002_010E']+df3['B15002_020E']+df3['B15002_021E']+df3['B15002_022E']+df3['B15002_023E']+df3['B15002_024E']+df3['B15002_025E']+df3['B15002_003E']
                                       +df3['B15002_026E']+df3['B15002_027E'])/df3['B15002_001E'])*100

In [23]:
df3['fips'] = df3['state code'] + df3['county code']
df3.head()

,county_name,B15002_003E,B15002_004E,B15002_005E,B15002_006E,B15002_007E,B15002_008E,B15002_009E,B15002_010E,B15002_020E,...,B15002_023E,B15002_024E,B15002_025E,B15002_026E,B15002_027E,B15002_001E,state code,county code,percent_no_highschool_diploma,fips
0,"Childress County, Texas",17,5,92,62,20,67,118,24,0,...,68,8,44,49,0,4806,48,075,13.462339,48075
1,"Comal County, Texas",607,333,472,458,405,490,626,400,658,...,542,301,586,349,626,82814,48,091,10.261550,48091
2,"Houston County, Texas",175,60,102,297,276,236,289,135,82,...,178,159,331,349,119,16706,48,225,18.556207,48225
3,"Navarro County, Texas",223,220,681,574,535,378,627,332,362,...,499,483,668,650,193,31308,48,349,23.367829,48349
4,"Scurry County, Texas",93,142,149,236,174,149,277,131,91,...,130,31,78,85,54,11171,48,415,19.541670,48415


#### 4.Percent of Population Living in Mobile Homes: (B25033_006E+B25033_007E+B25033_012E+B25033_013E)/B25033_001E

In [24]:
variables = 'B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E'
df4 = collectAll_county(year,variables)

['NAME', 'B25033_006E', 'B25033_007E', 'B25033_012E', 'B25033_013E', 'B25033_001E', 'state', 'county']


In [25]:
df4.shape

(3220, 8)

In [26]:
df4['B25033_006E'] = df4['B25033_006E'].apply(changeInteger)
df4['B25033_007E'] = df4['B25033_007E'].apply(changeInteger)
df4['B25033_012E'] = df4['B25033_012E'].apply(changeInteger)
df4['B25033_013E'] = df4['B25033_013E'].apply(changeInteger)
df4['B25033_001E'] = df4['B25033_001E'].apply(changeInteger)

In [27]:
df4['percent_mobile_homes'] = ((df4['B25033_006E'] + df4['B25033_007E'] + df4['B25033_012E'] + df4['B25033_013E'])/df4['B25033_001E'])*100

In [28]:
df4['fips'] = df4['state code'] + df4['county code']
df4.head()

,county_name,B25033_006E,B25033_007E,B25033_012E,B25033_013E,B25033_001E,state code,county code,percent_mobile_homes,fips
0,"Childress County, Texas",603,0,87,0,5823,48,075,11.849562,48075
1,"Comal County, Texas",8510,373,4384,54,118537,48,091,11.237841,48091
2,"Houston County, Texas",3331,42,1087,0,19714,48,225,22.623516,48225
3,"Navarro County, Texas",7218,45,2600,3,47273,48,349,20.870264,48349
4,"Scurry County, Texas",949,8,305,0,15455,48,415,8.165642,48415


#### 5.Percent of Population with No Vehicle Available: (B25044_003E+B25044_010E)/B25044_001E

In [29]:
variables = 'B25044_003E,B25044_010E,B25044_001E'
df5 = collectAll_county(year,variables)

['NAME', 'B25044_003E', 'B25044_010E', 'B25044_001E', 'state', 'county']


In [30]:
df5.shape

(3220, 6)

In [31]:
df5['B25044_003E'] = df5['B25044_003E'].apply(changeInteger)
df5['B25044_010E'] = df5['B25044_010E'].apply(changeInteger)
df5['B25044_001E'] = df5['B25044_001E'].apply(changeInteger)

In [32]:
df5['percent_no_vehicle'] = ((df5['B25044_003E'] + df5['B25044_010E'])/df5['B25044_001E'])*100

In [33]:
df5['fips'] = df5['state code'] + df5['county code']
df5.head()

,county_name,B25044_003E,B25044_010E,B25044_001E,state code,county code,percent_no_vehicle,fips
0,"Childress County, Texas",50,106,2391,48,075,6.524467,48075
1,"Comal County, Texas",579,768,43951,48,091,3.064777,48091
2,"Houston County, Texas",256,385,8046,48,225,7.966692,48225
3,"Navarro County, Texas",356,1123,17477,48,349,8.462551,48349
4,"Scurry County, Texas",41,85,5832,48,415,2.160494,48415


#### 6.Percent of Population Unemployed:sum(B21005_006E, B21005_007E, B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E)/B21005_001E

In [34]:
variables = 'B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E'
df6 = collectAll_county(year,variables)

['NAME', 'B21005_006E', 'B21005_007E', 'B21005_011E', 'B21005_012E', 'B21005_017E', 'B21005_022E', 'B21005_028E', 'B21005_029E', 'B21005_033E', 'B21005_034E', 'B21005_001E', 'state', 'county']


In [35]:
df6.shape

(3220, 14)

In [36]:
df6['B21005_006E'] = df6['B21005_006E'].apply(changeInteger)
df6['B21005_007E'] = df6['B21005_007E'].apply(changeInteger)
df6['B21005_011E'] = df6['B21005_011E'].apply(changeInteger)
df6['B21005_012E'] = df6['B21005_012E'].apply(changeInteger)
df6['B21005_017E'] = df6['B21005_017E'].apply(changeInteger)
df6['B21005_022E'] = df6['B21005_022E'].apply(changeInteger)
df6['B21005_028E'] = df6['B21005_028E'].apply(changeInteger)
df6['B21005_029E'] = df6['B21005_029E'].apply(changeInteger)
df6['B21005_033E'] = df6['B21005_033E'].apply(changeInteger)
df6['B21005_034E'] = df6['B21005_034E'].apply(changeInteger)
df6['B21005_001E'] = df6['B21005_001E'].apply(changeInteger)

In [37]:
df6['percent_unemployed_CDC'] = ((df6['B21005_006E']+df6['B21005_007E']+df6['B21005_011E']+df6['B21005_012E']+df6['B21005_017E']+df6['B21005_022E']+df6['B21005_028E']+df6['B21005_029E']+df6['B21005_033E']+df6['B21005_034E'])/df6['B21005_001E'])*100

In [38]:
df6['fips'] = df6['state code'] + df6['county code']
df6.head()

,county_name,B21005_006E,B21005_007E,B21005_011E,B21005_012E,B21005_017E,B21005_022E,B21005_028E,B21005_029E,B21005_033E,B21005_034E,B21005_001E,state code,county code,percent_unemployed_CDC,fips
0,"Childress County, Texas",0,28,63,1267,39,23,0,51,8,278,4614,48,075,38.079757,48075
1,"Comal County, Texas",20,144,1244,4733,148,769,78,983,336,5958,71486,48,091,20.161990,48091
2,"Houston County, Texas",15,69,234,1771,16,154,5,203,38,1435,13786,48,225,28.579719,48225
3,"Navarro County, Texas",0,26,1370,2081,3,761,43,393,148,1903,27877,48,349,24.134591,48349
4,"Scurry County, Texas",12,5,198,1488,0,43,0,109,0,508,10441,48,415,22.631932,48415


#### 7.Percent of Population Living In Accommodations with Less Than 1 Room Per Person/Crowding: (B25014_005E+B25014_006E+B25014_007E+B25014_011E+B25014_012E+B25014_013E)/B25014_001E

In [39]:
variables = 'B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E'
df7 = collectAll_county(year,variables)

['NAME', 'B25014_005E', 'B25014_006E', 'B25014_007E', 'B25014_011E', 'B25014_012E', 'B25014_013E', 'B25014_001E', 'state', 'county']


In [40]:
df7.shape

(3220, 10)

In [41]:
df7['B25014_005E'] = df7['B25014_005E'].apply(changeInteger)
df7['B25014_006E'] = df7['B25014_006E'].apply(changeInteger)
df7['B25014_007E'] = df7['B25014_007E'].apply(changeInteger)
df7['B25014_011E'] = df7['B25014_011E'].apply(changeInteger)
df7['B25014_012E'] = df7['B25014_012E'].apply(changeInteger)
df7['B25014_013E'] = df7['B25014_013E'].apply(changeInteger)
df7['B25014_001E'] = df7['B25014_001E'].apply(changeInteger)

In [42]:
df7['percent_overcrowding'] = ((df7['B25014_005E']+df7['B25014_006E']+df7['B25014_007E']+df7['B25014_011E']+df7['B25014_012E']+df7['B25014_013E'])/df7['B25014_001E'])*100

In [43]:
df7['fips'] = df7['state code'] + df7['county code']
df7.head()

,county_name,B25014_005E,B25014_006E,B25014_007E,B25014_011E,B25014_012E,B25014_013E,B25014_001E,state code,county code,percent_overcrowding,fips
0,"Childress County, Texas",0,0,0,21,0,0,2391,48,075,0.878294,48075
1,"Comal County, Texas",321,108,147,579,94,89,43951,48,091,3.044299,48091
2,"Houston County, Texas",87,0,12,39,9,23,8046,48,225,2.112851,48225
3,"Navarro County, Texas",421,71,43,266,52,18,17477,48,349,4.983693,48349
4,"Scurry County, Texas",77,42,0,93,15,41,5832,48,415,4.595336,48415


#### 8.Percent of Housing Units with 10+ Units in Structure: (B25024_007E+B25024_008E+B25024_009E)/B25024_001E

In [44]:
variables = 'B25024_007E,B25024_008E,B25024_009E,B25024_001E'
df8 = collectAll_county(year,variables)

['NAME', 'B25024_007E', 'B25024_008E', 'B25024_009E', 'B25024_001E', 'state', 'county']


In [45]:
df8.shape

(3220, 7)

In [46]:
df8['B25024_007E'] = df8['B25024_007E'].apply(changeInteger)
df8['B25024_008E'] = df8['B25024_008E'].apply(changeInteger)
df8['B25024_009E'] = df8['B25024_009E'].apply(changeInteger)
df8['B25024_001E'] = df8['B25024_001E'].apply(changeInteger)

In [47]:
df8['percent_multi_unit_housing'] = ((df8['B25024_007E']+df8['B25024_008E']+df8['B25024_009E'])/df8['B25024_001E'])*100

In [48]:
df8['fips'] = df8['state code'] + df8['county code']
df8.head()

,county_name,B25024_007E,B25024_008E,B25024_009E,B25024_001E,state code,county code,percent_multi_unit_housing,fips
0,"Childress County, Texas",110,42,122,3076,48,075,8.907672,48075
1,"Comal County, Texas",1295,960,832,50288,48,091,6.138641,48091
2,"Houston County, Texas",53,66,36,11543,48,225,1.342805,48225
3,"Navarro County, Texas",329,46,210,20431,48,349,2.863296,48349
4,"Scurry County, Texas",5,29,8,7137,48,415,0.588483,48415


#### 9.Percent Of Population 65+: (B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_049E)/B01001_001E

In [49]:
variables = 'B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E'
df9 = collectAll_county(year,variables)

['NAME', 'B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E', 'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E', 'B01001_001E', 'state', 'county']


In [50]:
df9.shape

(3220, 16)

In [51]:
df9['B01001_020E'] = df9['B01001_020E'].apply(changeInteger)
df9['B01001_021E'] = df9['B01001_021E'].apply(changeInteger)
df9['B01001_022E'] = df9['B01001_022E'].apply(changeInteger)
df9['B01001_023E'] = df9['B01001_023E'].apply(changeInteger)
df9['B01001_024E'] = df9['B01001_024E'].apply(changeInteger)
df9['B01001_025E'] = df9['B01001_025E'].apply(changeInteger)
df9['B01001_044E'] = df9['B01001_044E'].apply(changeInteger)
df9['B01001_045E'] = df9['B01001_045E'].apply(changeInteger)
df9['B01001_046E'] = df9['B01001_046E'].apply(changeInteger)
df9['B01001_047E'] = df9['B01001_047E'].apply(changeInteger)
df9['B01001_048E'] = df9['B01001_048E'].apply(changeInteger)
df9['B01001_049E'] = df9['B01001_049E'].apply(changeInteger)
df9['B01001_001E'] = df9['B01001_001E'].apply(changeInteger)

In [52]:
df9['percent_age_65_and_older'] = ((df9['B01001_020E']+df9['B01001_021E']+df9['B01001_022E']+df9['B01001_023E']+df9['B01001_024E']+df9['B01001_025E']+df9['B01001_044E']
                                  +df9['B01001_045E']+df9['B01001_046E']+df9['B01001_047E']+df9['B01001_048E']+df9['B01001_049E'])/df9['B01001_001E'])*100

In [53]:
df9['fips'] = df9['state code'] + df9['county code']
df9.head()

,county_name,B01001_020E,B01001_021E,B01001_022E,B01001_023E,B01001_024E,B01001_025E,B01001_044E,B01001_045E,B01001_046E,B01001_047E,B01001_048E,B01001_049E,B01001_001E,state code,county code,percent_age_65_and_older,fips
0,"Childress County, Texas",24,125,144,70,90,23,51,78,87,116,91,121,7059,48,075,14.449639,48075
1,"Comal County, Texas",1408,2136,2469,1404,995,1019,1613,1849,2875,1720,1344,1424,119632,48,091,16.931925,48091
2,"Houston County, Texas",285,427,515,431,265,220,253,383,659,443,371,413,22949,48,225,20.327683,48225
3,"Navarro County, Texas",477,756,923,457,454,301,606,755,977,593,548,714,48118,48,349,15.713454,48349
4,"Scurry County, Texas",188,241,210,208,111,127,141,304,277,211,260,160,17238,48,415,14.143172,48415


#### 10.Percent Of Population Who Do Not Speak English: B99163_005E/B99163_001E

In [54]:
variables = 'B99163_005E,B99163_001E'
df10 = collectAll_county(year,variables)

['NAME', 'B99163_005E', 'B99163_001E', 'state', 'county']


In [55]:
df10.shape

(3220, 5)

In [56]:
df10['B99163_005E'] = df10['B99163_005E'].apply(changeInteger)
df10['B99163_001E'] = df10['B99163_001E'].apply(changeInteger)

In [57]:
df10['percent_limited_english_abilities'] = (df10['B99163_005E']/df10['B99163_001E'])*100

In [58]:
df10['fips'] = df10['state code'] + df10['county code']
df10.head()

,county_name,B99163_005E,B99163_001E,state code,county code,percent_limited_english_abilities,fips
0,"Childress County, Texas",844,6720,48,075,12.559524,48075
1,"Comal County, Texas",18861,112867,48,091,16.710819,48091
2,"Houston County, Texas",1629,21768,48,225,7.483462,48225
3,"Navarro County, Texas",9249,44666,48,349,20.707025,48349
4,"Scurry County, Texas",4309,16000,48,415,26.931250,48415


#### 11.Percent of Population 17 Years of Age and Under: (B01001_003E+B01001_004E+B01001_005E+B01001_006E+B01001_027E+B01001_028E+B01001_029E+B01001_030E)/B01003_001E

In [59]:
variables = 'B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E'
df11 = collectAll_county(year,variables)

['NAME', 'B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E', 'B01001_028E', 'B01001_029E', 'B01001_030E', 'B01003_001E', 'state', 'county']


In [60]:
df11.shape

(3220, 12)

In [61]:
df11['B01001_003E'] = df11['B01001_003E'].apply(changeInteger)
df11['B01001_004E'] = df11['B01001_004E'].apply(changeInteger)
df11['B01001_005E'] = df11['B01001_005E'].apply(changeInteger)
df11['B01001_006E'] = df11['B01001_006E'].apply(changeInteger)
df11['B01001_027E'] = df11['B01001_027E'].apply(changeInteger)
df11['B01001_028E'] = df11['B01001_028E'].apply(changeInteger)
df11['B01001_029E'] = df11['B01001_029E'].apply(changeInteger)
df11['B01001_030E'] = df11['B01001_030E'].apply(changeInteger)
df11['B01003_001E'] = df11['B01003_001E'].apply(changeInteger)

In [62]:
df11['percent_age_17_and_younger'] = ((df11['B01001_003E']+df11['B01001_004E']+df11['B01001_005E']+df11['B01001_006E']+df11['B01001_027E']+df11['B01001_028E']+df11['B01001_029E']+df11['B01001_030E'])/df11['B01003_001E'])*100

In [63]:
df11['fips'] = df11['state code'] + df11['county code']
df11.head()

,county_name,B01001_003E,B01001_004E,B01001_005E,B01001_006E,B01001_027E,B01001_028E,B01001_029E,B01001_030E,B01003_001E,state code,county code,percent_age_17_and_younger,fips
0,"Childress County, Texas",112,113,243,152,227,282,226,70,7059,48,075,20.186995,48075
1,"Comal County, Texas",3467,3884,4382,2558,3298,3476,4146,2534,119632,48,091,23.191955,48091
2,"Houston County, Texas",584,625,655,387,597,521,726,384,22949,48,225,19.517190,48225
3,"Navarro County, Texas",1750,1752,1901,1154,1702,1670,1700,1024,48118,48,349,26.295773,48349
4,"Scurry County, Texas",660,676,536,355,578,477,708,369,17238,48,415,25.287156,48415


#### 12.Percent Minority: 1-(B03002_003E/B03002_001E)

In [64]:
variables = 'B03002_003E,B03002_001E'
df12 = collectAll_county(year,variables)

['NAME', 'B03002_003E', 'B03002_001E', 'state', 'county']


In [65]:
df12.shape

(3220, 5)

In [66]:
df12['B03002_003E'] = df12['B03002_003E'].apply(changeInteger)
df12['B03002_001E'] = df12['B03002_001E'].apply(changeInteger)

In [67]:
df12['percent_minorities'] = (1 - (df12['B03002_003E']/df12['B03002_001E']))*100

In [68]:
df12['fips'] = df12['state code'] + df12['county code']
df12.head()

,county_name,B03002_003E,B03002_001E,state code,county code,percent_minorities,fips
0,"Childress County, Texas",4542,7059,48,075,35.656609,48075
1,"Comal County, Texas",82977,119632,48,091,30.639795,48091
2,"Houston County, Texas",14212,22949,48,225,38.071376,48225
3,"Navarro County, Texas",28099,48118,48,349,41.603974,48349
4,"Scurry County, Texas",9494,17238,48,415,44.924005,48415


#### 13.Percent of Population with a Disability: (B18101_004E+B18101_007E+B18101_010E+B18101_013E+B18101_016E+B18101_019E)/B18101_002E + (B18101_023E+B18101_026E+B18101_029E+B18101_032E+B18101_035E+B18101_038E)/B18101_021E

In [69]:
variables = 'B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E'
df13 = collectAll_county(year,variables)

['NAME', 'B18101_004E', 'B18101_007E', 'B18101_010E', 'B18101_013E', 'B18101_016E', 'B18101_019E', 'B18101_002E', 'B18101_023E', 'B18101_026E', 'B18101_029E', 'B18101_032E', 'B18101_035E', 'B18101_038E', 'B18101_021E', 'state', 'county']


In [70]:
df13.shape

(3220, 17)

In [71]:
df13['B18101_004E'] = df13['B18101_004E'].apply(changeInteger)
df13['B18101_007E'] = df13['B18101_007E'].apply(changeInteger)
df13['B18101_010E'] = df13['B18101_010E'].apply(changeInteger)
df13['B18101_013E'] = df13['B18101_013E'].apply(changeInteger)
df13['B18101_016E'] = df13['B18101_016E'].apply(changeInteger)
df13['B18101_002E'] = df13['B18101_002E'].apply(changeInteger)
df13['B18101_023E'] = df13['B18101_023E'].apply(changeInteger)
df13['B18101_026E'] = df13['B18101_026E'].apply(changeInteger)
df13['B18101_029E'] = df13['B18101_029E'].apply(changeInteger)
df13['B18101_032E'] = df13['B18101_032E'].apply(changeInteger)
df13['B18101_035E'] = df13['B18101_035E'].apply(changeInteger)
df13['B18101_038E'] = df13['B18101_038E'].apply(changeInteger)
df13['B18101_021E'] = df13['B18101_021E'].apply(changeInteger)

In [72]:
df13['percent_disabled'] = ((df13['B18101_004E']+df13['B18101_007E']+df13['B18101_010E']+df13['B18101_013E']+df13['B18101_016E'])/df13['B18101_002E'] + (df13['B18101_023E']+df13['B18101_026E']+df13['B18101_029E']+df13['B18101_032E']+df13['B18101_035E']+df13['B18101_038E'])/df13['B18101_021E'])*100

In [73]:
df13['fips'] = df13['state code'] + df13['county code']
df13.head()

,county_name,B18101_004E,B18101_007E,B18101_010E,B18101_013E,B18101_016E,B18101_019E,B18101_002E,B18101_023E,B18101_026E,B18101_029E,B18101_032E,B18101_035E,B18101_038E,B18101_021E,state code,county code,percent_disabled,fips
0,"Childress County, Texas",0,20,0,201,69,119,2891,0,7,0,294,64,172,2932,48,075,28.346274,48075
1,"Comal County, Texas",37,683,933,3411,1448,1595,58675,31,496,423,3309,1622,2246,59991,48,091,24.645456,48091
2,"Houston County, Texas",0,79,65,690,363,431,9253,25,51,112,787,425,657,10513,48,225,32.502596,48225
3,"Navarro County, Texas",22,423,463,2035,954,586,23220,3,320,658,1654,879,1019,24162,48,349,35.543810,48349
4,"Scurry County, Texas",92,91,78,353,284,243,7881,40,54,177,344,206,256,7873,48,415,25.074158,48415


#### 14.Percent of Population Living in Group Quarters: B26001_001E/B01003_001E

In [74]:
variables = 'B26001_001E,B01003_001E'
df14 = collectAll_county(year,variables)

['NAME', 'B26001_001E', 'B01003_001E', 'state', 'county']


In [75]:
df14.shape

(3220, 5)

In [76]:
df14['B26001_001E'] = df14['B26001_001E'].apply(changeInteger)
df14['B01003_001E'] = df14['B01003_001E'].apply(changeInteger)

In [77]:
df14['percent_institutionalized_in_group_quarters'] = (df14['B26001_001E']/df14['B01003_001E'])*100

In [78]:
df14['fips'] = df14['state code'] + df14['county code']
df14.head()

,county_name,B26001_001E,B01003_001E,state code,county code,percent_institutionalized_in_group_quarters,fips
0,"Childress County, Texas",1236,7059,48,075,17.509562,48075
1,"Comal County, Texas",1095,119632,48,091,0.915307,48091
2,"Houston County, Texas",3235,22949,48,225,14.096475,48225
3,"Navarro County, Texas",845,48118,48,349,1.756100,48349
4,"Scurry County, Texas",1783,17238,48,415,10.343427,48415


#### 15.Percent of Children Living in Single Parent Households: (B09008_010E+B09008_011E+B09008_012E)/B09008_001E

In [79]:
variables = 'B09008_010E,B09008_011E,B09008_012E,B09008_001E'
df15 = collectAll_county(year,variables)

['NAME', 'B09008_010E', 'B09008_011E', 'B09008_012E', 'B09008_001E', 'state', 'county']


In [80]:
df15['B09008_010E'] = df15['B09008_010E'].apply(changeInteger)
df15['B09008_011E'] = df15['B09008_011E'].apply(changeInteger)
df15['B09008_012E'] = df15['B09008_012E'].apply(changeInteger)
df15['B09008_001E'] = df15['B09008_001E'].apply(changeInteger)

In [81]:
df15['percent_single_parent_households_CDC'] = ((df15['B09008_010E'] + df15['B09008_011E']+df15['B09008_012E'])/df15['B09008_001E'])*100

In [82]:
df15['fips'] = df15['state code'] + df15['county code']
df15.head()

,county_name,B09008_010E,B09008_011E,B09008_012E,B09008_001E,state code,county code,percent_single_parent_households_CDC,fips
0,"Childress County, Texas",0,345,0,1401,48,075,24.625268,48075
1,"Comal County, Texas",823,4427,0,27683,48,091,18.964708,48091
2,"Houston County, Texas",76,1428,3,4392,48,225,34.312386,48225
3,"Navarro County, Texas",532,2889,2,12490,48,349,27.405925,48349
4,"Scurry County, Texas",220,808,0,4359,48,415,23.583391,48415


### age 18 to 34
#### (B01001_007E+B01001_008E+B01001_009E+B01001_010E+B01001_011E+B01001_012E+B01001_031E+B01001_032E+B01001_033E+B01001_034E+B01001_035E+B01001_036E)/B01001_001E

In [83]:
variables = 'B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E'
df16 = collectAll_county(year,variables)

['NAME', 'B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E', 'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E', 'B01001_001E', 'state', 'county']


In [84]:
df16['B01001_007E'] = df16['B01001_007E'].apply(changeInteger)
df16['B01001_008E'] = df16['B01001_008E'].apply(changeInteger)
df16['B01001_009E'] = df16['B01001_009E'].apply(changeInteger)
df16['B01001_010E'] = df16['B01001_010E'].apply(changeInteger)
df16['B01001_011E'] = df16['B01001_011E'].apply(changeInteger)
df16['B01001_012E'] = df16['B01001_012E'].apply(changeInteger)
df16['B01001_031E'] = df16['B01001_031E'].apply(changeInteger)
df16['B01001_032E'] = df16['B01001_032E'].apply(changeInteger)
df16['B01001_033E'] = df16['B01001_033E'].apply(changeInteger)
df16['B01001_034E'] = df16['B01001_034E'].apply(changeInteger)
df16['B01001_035E'] = df16['B01001_035E'].apply(changeInteger)
df16['B01001_036E'] = df16['B01001_036E'].apply(changeInteger)
df16['B01001_001E'] = df16['B01001_001E'].apply(changeInteger)

In [85]:
df16['percent_age_18_to_34'] = ((df16['B01001_007E'] + df16['B01001_008E'] +df16['B01001_009E'] +df16['B01001_010E'] +df16['B01001_011E']+df16['B01001_012E']+df16['B01001_031E']+df16['B01001_032E']+df16['B01001_033E']+df16['B01001_034E']+df16['B01001_035E']+df16['B01001_036E'])/df16['B01001_001E'])*100

In [86]:
df16['fips'] = df16['state code'] + df16['county code']
df16.head()

,county_name,B01001_007E,B01001_008E,B01001_009E,B01001_010E,B01001_011E,B01001_012E,B01001_031E,B01001_032E,B01001_033E,B01001_034E,B01001_035E,B01001_036E,B01001_001E,state code,county code,percent_age_18_to_34,fips
0,"Childress County, Texas",141,38,45,418,537,442,70,0,16,100,149,201,7059,48,075,30.556736,48075
1,"Comal County, Texas",1501,484,821,1970,3122,3098,1278,507,459,2053,3152,3244,119632,48,091,18.129765,48091
2,"Houston County, Texas",220,201,60,513,643,742,195,170,40,365,528,544,22949,48,225,18.392958,48225
3,"Navarro County, Texas",669,408,413,754,1353,1213,643,193,294,783,1638,1354,48118,48,349,20.189950,48349
4,"Scurry County, Texas",208,180,103,456,651,738,275,168,77,241,429,536,17238,48,415,23.564219,48415


### Age 35 to 44
#### (B01001_013E+B01001_014E+B01001_037E+B01001_038E)/B01001_001E

In [87]:
variables = 'B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E'
df17 = collectAll_county(year,variables)

['NAME', 'B01001_013E', 'B01001_014E', 'B01001_037E', 'B01001_038E', 'B01001_001E', 'state', 'county']


In [88]:
df17['B01001_013E'] = df17['B01001_013E'].apply(changeInteger)
df17['B01001_014E'] = df17['B01001_014E'].apply(changeInteger)
df17['B01001_037E'] = df17['B01001_037E'].apply(changeInteger)
df17['B01001_038E'] = df17['B01001_038E'].apply(changeInteger)
df17['B01001_001E'] = df17['B01001_001E'].apply(changeInteger)

In [89]:
df17['percent_age_35_to_44'] = ((df17['B01001_013E']+df17['B01001_014E'] +df17['B01001_037E']+df17['B01001_038E'])/df17['B01001_001E'])*100

In [90]:
df17['fips'] = df17['state code'] + df17['county code']
df17.head()

,county_name,B01001_013E,B01001_014E,B01001_037E,B01001_038E,B01001_001E,state code,county code,percent_age_35_to_44,fips
0,"Childress County, Texas",262,217,342,111,7059,48,075,13.203003,48075
1,"Comal County, Texas",3327,3745,3148,4218,119632,48,091,12.068677,48091
2,"Houston County, Texas",820,996,668,448,22949,48,225,12.776156,48225
3,"Navarro County, Texas",1103,1843,1487,1409,48118,48,349,12.140987,48349
4,"Scurry County, Texas",735,533,457,371,17238,48,415,12.159183,48415


### age 45 to 54
#### (B01001_015E+B01001_016E+B01001_039E+B01001_040E)/B01001_001E

In [91]:
variables = 'B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E'
df18 = collectAll_county(year,variables)

['NAME', 'B01001_015E', 'B01001_016E', 'B01001_039E', 'B01001_040E', 'B01001_001E', 'state', 'county']


In [92]:
df18['B01001_015E'] = df18['B01001_015E'].apply(changeInteger)
df18['B01001_016E'] = df18['B01001_016E'].apply(changeInteger)
df18['B01001_039E'] = df18['B01001_039E'].apply(changeInteger)
df18['B01001_040E'] = df18['B01001_040E'].apply(changeInteger)
df18['B01001_001E'] = df18['B01001_001E'].apply(changeInteger)

In [93]:
df18['percent_age_45_to_54'] = ((df18['B01001_015E']+df18['B01001_016E'] +df18['B01001_039E']+df18['B01001_040E'])/df18['B01001_001E'])*100

In [94]:
df18['fips'] = df18['state code'] + df18['county code']
df18.head()

,county_name,B01001_015E,B01001_016E,B01001_039E,B01001_040E,B01001_001E,state code,county code,percent_age_45_to_54,fips
0,"Childress County, Texas",168,282,103,218,7059,48,075,10.922227,48075
1,"Comal County, Texas",4240,4441,4200,4766,119632,48,091,14.751070,48091
2,"Houston County, Texas",1027,1003,613,797,22949,48,225,14.989760,48225
3,"Navarro County, Texas",1526,1637,1585,1695,48118,48,349,13.390000,48349
4,"Scurry County, Texas",738,607,493,466,17238,48,415,13.365820,48415


### age 55 to 64
#### (B01001_017E+B01001_018E+B01001_019E+B01001_041E+B01001_042E+B01001_043E)/B01001_001E

In [95]:
variables = 'B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E'
df19 = collectAll_county(year,variables)

['NAME', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_041E', 'B01001_042E', 'B01001_043E', 'B01001_001E', 'state', 'county']


In [96]:
df19['B01001_017E'] = df19['B01001_017E'].apply(changeInteger)
df19['B01001_018E'] = df19['B01001_018E'].apply(changeInteger)
df19['B01001_019E'] = df19['B01001_019E'].apply(changeInteger)
df19['B01001_041E'] = df19['B01001_041E'].apply(changeInteger)
df19['B01001_042E'] = df19['B01001_042E'].apply(changeInteger)
df19['B01001_043E'] = df19['B01001_043E'].apply(changeInteger)
df19['B01001_001E'] = df19['B01001_001E'].apply(changeInteger)

In [97]:
df19['percent_age_55_to_64'] = ((df19['B01001_017E']+df19['B01001_018E'] +df19['B01001_019E']+df19['B01001_041E']+df19['B01001_042E']+df19['B01001_043E'])/df19['B01001_001E'])*100

In [98]:
df19['fips'] = df19['state code'] + df19['county code']
df19.head()

,county_name,B01001_017E,B01001_018E,B01001_019E,B01001_041E,B01001_042E,B01001_043E,B01001_001E,state code,county code,percent_age_55_to_64,fips
0,"Childress County, Texas",248,57,99,226,50,74,7059,48,075,10.681400,48075
1,"Comal County, Texas",4127,2085,2504,4770,1721,2650,119632,48,091,14.926608,48091
2,"Houston County, Texas",985,218,405,806,322,476,22949,48,225,13.996253,48225
3,"Navarro County, Texas",1501,399,973,1495,521,1015,48118,48,349,12.269837,48349
4,"Scurry County, Texas",559,203,289,392,219,317,17238,48,415,11.480450,48415


### Combine 19 dataframes together

In [99]:
df = df1[['fips','per_capita_income']].merge(df2[['fips', 'percent_below_poverty']], on = 'fips', how = 'left')

In [100]:
df = df.merge(df3[['fips', 'percent_no_highschool_diploma']], on = 'fips', how = 'left')
df = df.merge(df4[['fips', 'percent_mobile_homes']], on = 'fips', how = 'left')
df = df.merge(df5[['fips', 'percent_no_vehicle']], on = 'fips', how = 'left')
df = df.merge(df6[['fips', 'percent_unemployed_CDC']], on = 'fips', how = 'left')
df = df.merge(df7[['fips', 'percent_overcrowding']], on = 'fips', how = 'left')
df = df.merge(df8[['fips', 'percent_multi_unit_housing']], on = 'fips', how = 'left')
df = df.merge(df9[['fips', 'percent_age_65_and_older']], on = 'fips', how = 'left')
df = df.merge(df10[['fips', 'percent_limited_english_abilities']], on = 'fips', how = 'left')
df = df.merge(df11[['fips', 'percent_age_17_and_younger']], on = 'fips', how = 'left')
df = df.merge(df12[['fips', 'percent_minorities']], on = 'fips', how = 'left')
df = df.merge(df13[['fips', 'percent_disabled']], on = 'fips', how = 'left')
df = df.merge(df14[['fips', 'percent_institutionalized_in_group_quarters']], on = 'fips', how = 'left')
df = df.merge(df15[['fips', 'percent_single_parent_households_CDC']], on = 'fips', how = 'left')
df = df.merge(df16[['fips', 'percent_age_18_to_34']], on = 'fips', how = 'left')
df = df.merge(df17[['fips', 'percent_age_35_to_44']], on = 'fips', how = 'left')
df = df.merge(df18[['fips', 'percent_age_45_to_54']], on = 'fips', how = 'left')
df = df.merge(df19[['fips', 'percent_age_55_to_64']], on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64
0,48075,19861,20.212949,13.462339,11.849562,6.524467,38.079757,0.878294,8.907672,14.449639,12.559524,20.186995,35.656609,28.346274,17.509562,24.625268,30.556736,13.203003,10.922227,10.681400
1,48091,32838,8.802986,10.261550,11.237841,3.064777,20.161990,3.044299,6.138641,16.931925,16.710819,23.191955,30.639795,24.645456,0.915307,18.964708,18.129765,12.068677,14.751070,14.926608
2,48225,17570,25.011443,18.556207,22.623516,7.966692,28.579719,2.112851,1.342805,20.327683,7.483462,19.517190,38.071376,32.502596,14.096475,34.312386,18.392958,12.776156,14.989760,13.996253
3,48349,20697,20.739015,23.367829,20.870264,8.462551,24.134591,4.983693,2.863296,15.713454,20.707025,26.295773,41.603974,35.543810,1.756100,27.405925,20.189950,12.140987,13.390000,12.269837
4,48415,24584,9.857973,19.541670,8.165642,2.160494,22.631932,4.595336,0.588483,14.143172,26.931250,25.287156,44.924005,25.074158,10.343427,23.583391,23.564219,12.159183,13.365820,11.480450


In [101]:
df['percent_age_18_to_64'] = 100 - df['percent_age_17_and_younger'] - df['percent_age_65_and_older']

In [102]:
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,...,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64,percent_age_18_to_64
0,48075,19861,20.212949,13.462339,11.849562,6.524467,38.079757,0.878294,8.907672,14.449639,...,20.186995,35.656609,28.346274,17.509562,24.625268,30.556736,13.203003,10.922227,10.681400,65.363366
1,48091,32838,8.802986,10.261550,11.237841,3.064777,20.161990,3.044299,6.138641,16.931925,...,23.191955,30.639795,24.645456,0.915307,18.964708,18.129765,12.068677,14.751070,14.926608,59.876120
2,48225,17570,25.011443,18.556207,22.623516,7.966692,28.579719,2.112851,1.342805,20.327683,...,19.517190,38.071376,32.502596,14.096475,34.312386,18.392958,12.776156,14.989760,13.996253,60.155127
3,48349,20697,20.739015,23.367829,20.870264,8.462551,24.134591,4.983693,2.863296,15.713454,...,26.295773,41.603974,35.543810,1.756100,27.405925,20.189950,12.140987,13.390000,12.269837,57.990773
4,48415,24584,9.857973,19.541670,8.165642,2.160494,22.631932,4.595336,0.588483,14.143172,...,25.287156,44.924005,25.074158,10.343427,23.583391,23.564219,12.159183,13.365820,11.480450,60.569672


In [103]:
df = df.sort_values('fips')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,...,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,percent_age_18_to_34,percent_age_35_to_44,percent_age_45_to_54,percent_age_55_to_64,percent_age_18_to_64
2135,01001,24974,12.879382,13.868774,16.469707,4.461659,19.230769,2.436752,3.485077,13.541950,...,25.432354,24.217236,32.116741,0.854747,24.009407,20.973905,13.972945,14.800529,11.278318,61.025697
2579,01003,27317,13.411661,10.780943,13.364355,2.984994,22.525936,1.076865,18.327977,18.130288,...,22.366122,16.896182,24.158805,1.425269,22.369328,19.074830,12.621399,13.941093,13.866268,59.503590
2126,01005,16824,26.727439,27.908337,27.502078,9.336370,35.787539,2.006072,1.498730,16.058963,...,21.305510,53.757612,42.272485,10.663894,50.174277,23.228873,12.475865,14.046487,12.884301,62.635527
2127,01007,18431,16.795878,20.474758,28.095468,5.293867,31.025161,0.227693,1.638613,14.400106,...,21.761635,25.499912,27.039354,8.246328,26.885246,22.748186,15.240665,13.851531,11.997876,63.838259
2133,01009,20532,16.720933,22.244214,24.436821,3.699078,26.333343,1.724635,0.905281,16.482412,...,23.711662,12.146942,29.502272,1.003292,23.549638,19.729683,12.900710,14.288685,12.886848,59.805926


In [104]:
df = df.drop(['percent_age_17_and_younger'], axis = 1)

In [105]:
df.to_csv('Social_Economic_2015.csv')

In [87]:
suicide_2017.head()

,county,fips,SuicideDeathRate
0,"Abbeville County, SC",45001,17.443705
1,"Acadia Parish, LA",22001,15.976993
2,"Accomack County, VA",51001,13.618293
3,"Ada County, ID",16001,18.824601
4,"Adair County, IA",19001,11.463289


In [88]:
df = df.merge(suicide_2017, on = 'fips', how = 'left')
df.head()

,fips,per_capita_income,percent_below_poverty,percent_no_highschool_diploma,percent_mobile_homes,percent_no_vehicle,percent_unemployed_CDC,percent_overcrowding,percent_multi_unit_housing,percent_age_65_and_older,percent_limited_english_abilities,percent_age_17_and_younger,percent_minorities,percent_disabled,percent_institutionalized_in_group_quarters,percent_single_parent_households_CDC,county,SuicideDeathRate
0,40045,27272,0.171555,0.124121,0.141763,0.028923,0.225352,0.010462,0.005712,0.209410,0.045809,0.236714,0.130912,0.481632,0.012921,0.285273,"Ellis County, OK",22.410690
1,40051,27334,0.124159,0.119022,0.145647,0.033480,0.208998,0.021679,0.023391,0.151570,0.037375,0.244448,0.180015,0.617015,0.012640,0.219735,"Grady County, OK",22.775042
2,40119,22785,0.263145,0.089728,0.100425,0.056900,0.321887,0.033154,0.111002,0.116006,0.062704,0.193678,0.226592,0.529113,0.099003,0.218812,"Payne County, OK",14.710389
3,40125,22284,0.169700,0.135085,0.122731,0.052806,0.243826,0.024648,0.032731,0.155835,0.043118,0.243011,0.270296,0.647418,0.045424,0.256268,"Pottawatomie County, OK",16.614516
4,40037,24056,0.151820,0.137377,0.208241,0.036480,0.227509,0.035571,0.024049,0.167859,0.026556,0.242246,0.232415,0.494830,0.011721,0.262649,"Creek County, OK",13.946223


In [89]:
df.shape

(3220, 18)

In [90]:
df.to_csv('Social_Economic_Suicide_2017.csv')